In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from statistics import stdev
import statsmodels
from statsmodels.sandbox.stats.multicomp import multipletests
import imblearn
import sys
import imblearn
from numpy import mean
from numpy import absolute
from numpy import sqrt
import scipy.stats
import sklearn.linear_model
import sklearn.neighbors
import pandas as pd
import sklearn.preprocessing
from xgboost import XGBClassifier
import sklearn.ensemble
from sklearn.model_selection import train_test_split
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from statistics import stdev
from scipy import stats
import statsmodels
from statsmodels.sandbox.stats.multicomp import multipletests
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import roc_curve, auc
import scipy.stats
import sklearn.linear_model
import sklearn.neighbors
from sklearn.naive_bayes import GaussianNB
from itertools import combinations

import sklearn.ensemble
from itertools import cycle
import time
from sklearn.metrics import RocCurveDisplay


# Define features

In [2]:
list_features = ['AK1C1','TAU', '1433G', 'SCUB1', 'FMOD', 'AMYP', 'CRIS3', 
                'MYDGF', 'RARR2', 'ATS8', 'PGK1', '1433Z', 'SV2A', 'TRH', 'GUAD', 
                'HV69D', 'CO7', 'SERC']

list_features_plus_biomarkers = ['AK1C1','TAU', '1433G', 'SCUB1', 'FMOD', 'AMYP', 'CRIS3', 
                'MYDGF', 'RARR2', 'ATS8', 'PGK1', '1433Z', 'SV2A', 'TRH', 'GUAD', 
                'HV69D', 'CO7', 'SERC', 'AB42/AB40', 'Ttau (pg/ml)']

list_features_plus_ab42 = ['AK1C1','TAU', '1433G', 'SCUB1', 'FMOD', 'AMYP', 'CRIS3', 
                'MYDGF', 'RARR2', 'ATS8', 'PGK1', '1433Z', 'SV2A', 'TRH', 'GUAD', 
                'HV69D', 'CO7', 'SERC', 'AB42/AB40']

list_features_plus_tau = ['AK1C1','TAU', '1433G', 'SCUB1', 'FMOD', 'AMYP', 'CRIS3', 
                'MYDGF', 'RARR2', 'ATS8', 'PGK1', '1433Z', 'SV2A', 'TRH', 'GUAD', 
                'HV69D', 'CO7', 'SERC', 'Ttau (pg/ml)']


list_features_ab42 = ['AB42/AB40']

list_features_tau = ['Ttau (pg/ml)']

list_features_both = ['AB42/AB40', 'Ttau (pg/ml)']

feature_options=[list_features, list_features_plus_ab42, list_features_plus_tau, 
                 list_features_plus_biomarkers, 
                 list_features_ab42, list_features_tau, list_features_both]

classes_of_interest=['Healthy', 'PD_MCI_LBD', 'PD', 'AD_MCI']


# Functions I used
I copied and pasted the functions I needed directly in this notebook-- but once this is finalized, I would import them instead.

In [3]:
def handle_scale_and_nan(df, nan_decision='drop'):
    features = list(df.select_dtypes(include='float64'))
    cat = list(df.select_dtypes(include='object'))
    scaler = sklearn.preprocessing.StandardScaler().fit(df[features])
    df_cont = pd.DataFrame(data=scaler.transform(df[features]), columns=features)
    df_cat = pd.DataFrame(data=df[cat], columns=cat)
    
    df = pd.concat([df_cat,df_cont],axis=1)
    
    if nan_decision == 'mean':
        for feature in features:
            df[feature].fillna((df[feature].mean()), inplace=True)
    elif nan_decision == 'drop':
            df = df.dropna(axis=1)
    elif nan_decision == 'impute':
        imputer = missingpy.MissForest() #must be in shape of n_samples by n_features
        df = imputer.fit_transform(df[:, 1:]) # impute NaNs with existing numerical vals
        df.insert(0, "group", df[:, 0], allow_duplicates=True) # reinsert the nominal vals
    elif nan_decision == 'replace_ones': 
        df.fillna(value=1)
        
    return df

def over_under(df_train,cat_in_excess='Healthy',target='group',randomstate=np.random.randint(0,4294967295)):
    """
    Takes dataframe(s) with only the target value and float64 features
    This function is to balance the samples in an imbalanced training dataset that has one category in excess, with additional categories more near each other
    The categories below the category in excess will be oversampled to equality, then the category in excess will be undersampled to equality
    ---Parameters---
    df_train: the training dataframe
    cat_in_excess: the category which is present in excess, far above the other categories
    target: target column in the dataframe
    randomstate: if chosen, this will the random state for the sampling. Default: None, numpy random integer method between 0 and 4294967295, the range of the sampling module used
    randomstate_sampler: the number of loops to run to compare random states starting from 
    """
        
    # Drop the excessive category and oversample minority to the intermediate category
    df_train_no_excess = df_train[df_train.group != cat_in_excess]
    over_sampler = imblearn.over_sampling.RandomOverSampler(random_state=randomstate)
    X_train = df_train_no_excess.drop(columns=target)
    y_train = df_train_no_excess[target]
    X_train_over, y_train_over = over_sampler.fit_resample(X_train,y_train)
    df_train_over = pd.concat([y_train_over,X_train_over],axis=1)

    # Re-introduce the excessive category and undersample the majority to the minority
    df_train_excess = pd.concat([df_train_over,df_train[df_train[target] == cat_in_excess]])
    under_sampler = imblearn.under_sampling.RandomUnderSampler(random_state=randomstate)
    X_train = df_train_excess.drop(columns=target)
    y_train = df_train_excess[target]
    X_train_under, y_train_under = under_sampler.fit_resample(X_train,y_train)
    df_train_eq = pd.concat([y_train_under,X_train_under],axis=1)
    
    return df_train_eq

def apply_ml_model(dev,classifier,scoring_method='balanced_accuracy',target='group', cv=10, feature_list=list_features):
    """
    Finds the score for different ML classifiers
    Takes a dataframe with only target and feature columns
    dev : the development dataframe with a single categorical target column and float64 features
    classifier: ML classifier to test.
        options: "random_forest', "naive_bayes", "decision_tree"
    scoring_method: method of scoring the classifier
        run sklearn.metrics.get_scorer_names() to get a list of scoring methods
    target: target categorical column
    cv: folds to run in evaluation. takes integers or 'max': will run maximum number of folds = # of samples in categories
    """
    # define predictor and response variables
    X = dev[feature_list]
    y = dev[target]
    
    # check that the data has equal number of categories in training data
    counts_list = list(dev['group'].value_counts())
    assert len(set(counts_list)) == 1, 'training data should contain equal quantities of categories. run bp_preprocessing.over_under() or other balancer'
    
    
    # choose model based on user input
    if classifier == "random_forest": 
        model = sklearn.ensemble.RandomForestClassifier()
    elif classifier == "naive_bayes":  
        model = sklearn.naive_bayes.GaussianNB()
    elif classifier == "decision_tree": 
        model = sklearn.tree.DecisionTreeClassifier()
    elif classifier == "xgboost": 
        model = XGBClassifier()
    else: 
        print("wrong classifier named entered")
    #define cv quantity:
    if type(cv) == int:
        pass
    elif cv == 'max':
        cv = counts_list[0]
    else:
        raise TypeError('Enter an integer or "max" as a string')
        
    scores = cross_val_score(model, X, y, scoring=scoring_method,cv=cv)
    mean_score = np.mean(absolute(scores))
    std = np.std(scores)
    stats_list = [cv, scores, mean_score, std , model, scoring_method, feature_list[-2:]]
    stats_df = pd.DataFrame(data=[stats_list], columns=['folds','scores','abs_avg_score','std','model','scoring_method', 'features'])
    
    return stats_df

In [4]:
def add_biomarkers_as_features(compiled_metadata, data_dev, data_test): 

    compiled_metadata = compiled_metadata.set_index("Public Sample ID")
    compiled_metadata = compiled_metadata.sort_index()
    compiled_metadata = compiled_metadata.dropna(axis = 0, how = 'all')
    compiled_metadata = compiled_metadata[(compiled_metadata.Condition != 'Golden West CSF Batch QC')]
    compiled_metadata = compiled_metadata[compiled_metadata.Condition != 'Lumbar CSF Pool Batch Ref']

    compiled_metadata = compiled_metadata[['Condition', 'Ttau (pg/ml)', 'AB42 (pg/ml)']]
    
    dict_AB42_ratio = {}
    for i, j in zip(compiled_metadata.index, compiled_metadata['AB42 (pg/ml)']): 
        dict_AB42_ratio[i]= j

    dict_tau = {}
    for i, j in zip(compiled_metadata.index, compiled_metadata['Ttau (pg/ml)']): 
        dict_tau[i]= j

    data_dev['AB42/AB40'] = data_dev['assay_ID'].map(dict_AB42_ratio)
    data_dev['Ttau (pg/ml)'] = data_dev['assay_ID'].map(dict_tau)

    data_test['AB42/AB40'] = data_test['assay_ID'].map(dict_AB42_ratio)
    data_test['Ttau (pg/ml)'] = data_test['assay_ID'].map(dict_tau)
    
    return data_dev, data_test

In [5]:
def pre_process_data(data_dev, data_test): 
    data_dev = handle_scale_and_nan(data_dev)
    data_test = handle_scale_and_nan(data_test)
    data_dev = data_dev.drop(columns='assay_ID')
    data_test = data_test.drop(columns='assay_ID')
    data_dev = over_under(data_dev) #split groups equally 
    return data_dev, data_test

def make_confusion_mtrx(dev, data_test, features=list_features):
    test_X = data_test[list_features]
    test_y = data_test['group']

    dev_X = dev[list_features]
    dev_y = dev.iloc[:,0] # 0th column is our target

    model = sklearn.ensemble.RandomForestClassifier()
    model.fit(dev_X, dev_y) 
    print('score=', sklearn.metrics.balanced_accuracy_score(test_y, model.predict(test_X)))
    ConfusionMatrixDisplay.from_estimator(model, test_X, test_y)
    plt.show()

In [6]:
def roc_curves_one_vs_rest(data_dev, data_test, list_features=list_features_plus_biomarkers):
    data_dev, data_test = pre_process_data(data_dev, data_test)
    
    dev = data_dev[list_features]
    dev.insert(0, "group", data_dev['group'], True)
    
    X_test = data_test[list_features]
    y_test = data_test['group']

    X_train = dev.iloc[:,1:] 
    y_train = dev.iloc[:,0] # 0th column is our target


    classifier = sklearn.ensemble.RandomForestClassifier()
    y_score = classifier.fit(X_train, y_train).predict_proba(X_test)

    label_binarizer = LabelBinarizer().fit(y_train)
    y_onehot_test = label_binarizer.transform(y_test)

    fig, ax = plt.subplots(figsize=(6, 6))
    
    # store the fpr, tpr, and roc_auc for all averaging strategies
    fpr, tpr, roc_auc = dict(), dict(), dict()
    
    # Compute micro-average ROC curve and ROC area
    fpr["micro"], tpr["micro"], _ = roc_curve(y_onehot_test.ravel(), y_score.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
    
    plt.plot(
        fpr["micro"],
        tpr["micro"],
        label=f"micro-average ROC curve (AUC = {roc_auc['micro']:.2f})",
        color="deeppink",
        linestyle=":",
        linewidth=4,
    )

    colors = cycle(["aqua", "darkorange", "cornflowerblue", 'red'])
    for class_id, color in zip(range(4), colors):
        RocCurveDisplay.from_predictions(
            y_onehot_test[:, class_id],
            y_score[:, class_id],
            name=f"ROC curve for {classes_of_interest[class_id]}",
            color=color,
            ax=ax,
        )

    plt.plot([0, 1], [0, 1], "k--", label="ROC curve for chance level (AUC = 0.5)")
    plt.axis("square")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    if len(list_features)==20: 
        plt.title(f"One-vs-Rest ROC curves, using 18 features + {list_features[-2:]}")
    elif len(list_features)==18:
        plt.title(f"One-vs-Rest ROC curves, using 18 features")
    elif len(list_features)==19:
        plt.title(f"One-vs-Rest ROC curves, using 18 features + {list_features[-1:]}")
    else: 
        plt.title(f"One-vs-Rest ROC curves, using {list_features}")
        
    plt.legend()
    plt.show()

In [7]:
def roc_curve_one_vs_one(data_dev, data_test, list_features): 
    pair_scores = []
    mean_tpr = dict()
    pair_list = list(combinations(np.unique(classes_of_interest), 2))
    
    data_dev, data_test = pre_process_data(data_dev, data_test)
    dev = data_dev[list_features]
    dev.insert(0, "group", data_dev['group'], True)
    
    X_test = data_test[list_features]
    y_test = data_test['group']

    X_train = dev.iloc[:,1:] 
    y_train = dev.iloc[:,0] # 0th column is our target


    classifier = sklearn.ensemble.RandomForestClassifier()
    y_score = classifier.fit(X_train, y_train).predict_proba(X_test)
    
    label_binarizer = LabelBinarizer().fit(y_train)
    
    for ix, (label_a, label_b) in enumerate(pair_list):

        a_mask = y_test == label_a
        b_mask = y_test == label_b
        ab_mask = np.logical_or(a_mask, b_mask)

        a_true = a_mask[ab_mask]
        b_true = b_mask[ab_mask]

        idx_a = np.flatnonzero(label_binarizer.classes_ == label_a)[0]
        idx_b = np.flatnonzero(label_binarizer.classes_ == label_b)[0]

        fpr_a, tpr_a, _ = roc_curve(a_true, y_score[ab_mask, idx_a])
        fpr_b, tpr_b, _ = roc_curve(b_true, y_score[ab_mask, idx_b])

        fpr_grid = np.linspace(0.0, 1.0, 1000)

        mean_tpr[ix] = np.zeros_like(fpr_grid)
        mean_tpr[ix] += np.interp(fpr_grid, fpr_a, tpr_a)
        mean_tpr[ix] += np.interp(fpr_grid, fpr_b, tpr_b)
        mean_tpr[ix] /= 2
        mean_score = auc(fpr_grid, mean_tpr[ix])
        pair_scores.append(mean_score)

        fig, ax = plt.subplots(figsize=(6, 6))
        plt.plot(
            fpr_grid,
            mean_tpr[ix],
            label=f"Mean {label_a} vs {label_b} (AUC = {mean_score :.2f})",
            linestyle=":",
            linewidth=4,
        )
        RocCurveDisplay.from_predictions(
            a_true,
            y_score[ab_mask, idx_a],
            ax=ax,
            name=f"{label_a} as positive class",
        )
        RocCurveDisplay.from_predictions(
            b_true,
            y_score[ab_mask, idx_b],
            ax=ax,
            name=f"{label_b} as positive class",
        )
        plt.plot([0, 1], [0, 1], "k--", label="chance level (AUC = 0.5)")
        plt.axis("square")
        plt.xlabel("False Positive Rate")
        plt.ylabel("True Positive Rate")

        if len(list_features)==20: 
            plt.title(f"{label_a} vs {label_b} ROC curves, using 18 features + {list_features[-2:]}")
        elif len(list_features)==18:
            plt.title(f"{label_a} vs {label_b} ROC curves, using 18 features")
        elif len(list_features)==19:
            plt.title(f"{label_a} vs {label_b} ROC curves, using 18 features + {list_features[-1:]}")
        else: 
            plt.title(f"{label_a} vs {label_b} ROC curves, using {list_features}")
        
            
        plt.legend()
        plt.show()


# Importing data
Import dev and test data from split_data directory. 

In [8]:
data_dev = pd.read_csv('/Users/mariamb/Desktop/BrainPower/brainpower/data/split_data/dev.csv')
data_test = pd.read_csv('/Users/mariamb/Desktop/BrainPower/brainpower/data/split_data/test.csv')
metadata = pd.read_csv('/Users/mariamb/Desktop/BrainPower/brainpower/data/metadata.csv')

data_dev, data_test = add_biomarkers_as_features(metadata, data_dev, data_test)

data_dev, data_test = pre_process_data(data_dev, data_test)

dev = data_dev[list_features_plus_biomarkers]
dev.insert(0, "group", data_dev['group'], True)
dev

,group,AK1C1,TAU,1433G,SCUB1,FMOD,AMYP,CRIS3,MYDGF,RARR2,...,PGK1,1433Z,SV2A,TRH,GUAD,HV69D,CO7,SERC,AB42/AB40,Ttau (pg/ml)
0,AD_MCI,-0.494309,-0.846909,0.899597,-0.411631,0.111899,0.620927,-0.120125,-0.763607,-0.840882,...,0.598239,0.812226,0.052515,-2.927328,1.790729,-2.361951,-1.099094,-0.145335,-0.800179,0.620095
1,AD_MCI,-0.400519,1.138241,1.379783,0.074543,1.463598,-0.472183,-2.027509,-0.880584,0.619631,...,1.005253,1.304040,-0.437144,-0.480640,1.084290,1.385088,-0.643078,0.074853,-1.194905,0.612778
2,AD_MCI,1.477217,0.363325,-0.206424,0.341163,0.434413,-0.292046,0.022677,0.322811,0.941360,...,-0.586312,0.312109,-0.630677,-1.548278,-1.212487,0.721655,-0.125949,-1.028790,-0.922864,-0.400673
3,AD_MCI,-0.540071,1.850655,2.466744,1.066619,-0.729014,0.350433,-0.440646,0.627074,-0.382661,...,2.430789,2.351507,-0.171098,-0.221895,-0.157784,0.015305,0.377590,1.676169,-0.824182,1.417685
4,AD_MCI,-0.025974,-0.712422,0.880565,-0.992573,-0.468207,0.753878,-1.324054,1.313589,-0.524040,...,1.011173,0.638534,0.937442,1.206685,2.540706,-0.289008,-1.112227,1.196369,-0.326241,0.223861
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,PD_MCI_LBD,-0.823057,-0.200307,0.758423,0.048004,0.263547,-1.371947,0.585728,3.599443,1.269535,...,1.576836,-0.086893,-0.150382,0.022824,1.056651,0.803522,-0.419239,0.137968,0.404536,-0.570070
168,PD_MCI_LBD,-0.253150,1.698337,0.520508,0.360982,1.162127,0.311244,1.255747,1.566336,1.376595,...,1.427066,0.883562,1.311786,0.150299,0.481897,-0.274526,1.669666,0.133994,-1.113026,2.298326
169,PD_MCI_LBD,-1.521499,0.041839,0.650621,-0.099473,-2.140942,-1.771112,0.620966,-1.443248,-0.200206,...,-1.311166,0.237034,-0.144658,0.495978,-0.018277,0.334649,1.224290,-2.014594,-0.006193,0.201909
170,PD_MCI_LBD,-0.133024,-0.418209,-0.057166,-0.654956,0.883038,0.043402,-0.075012,0.523597,0.965335,...,0.245195,-0.202206,0.457010,0.319129,0.255220,-0.283393,0.534689,1.489452,-0.814314,-0.350550


# Testing different ML models 
Here I am running the "applying_ml_model.py" code as a way to test which classifiers are best

Results: Naive bayes = around .5, random_forest= around .66, decision_tree = around .60

In [9]:
apply_ml_model(dev, 'random_forest', feature_list=list_features)

,folds,scores,abs_avg_score,std,model,scoring_method,features
0,10,"[0.7625, 0.5750000000000001, 0.7625, 0.6875, 0...",0.72,0.106975,RandomForestClassifier(),balanced_accuracy,"[CO7, SERC]"


In [ ]:
apply_ml_model(dev, 'naive_bayes', feature_list=list_features)

In [ ]:
apply_ml_model(dev, 'decision_tree', feature_list=list_features_plus_biomarkers)

# Running on the entire dev and test set. 

# Using 18 features

In [ ]:
make_confusion_mtrx(dev, data_test, features=list_features)

# Using 18 features + Tau and AB42/40

In [ ]:
make_confusion_mtrx(dev, data_test, features=list_features_plus_biomarkers)

# Using 18 features + AB42/40

In [ ]:
make_confusion_mtrx(dev, data_test, features=list_features_plus_ab42)

In [ ]:
test_X = data_test[list_features]
test_y = data_test['group']

dev_X = dev[list_features]
dev_y = dev.iloc[:,0] # 0th column is our target

model = sklearn.ensemble.RandomForestClassifier()
model.fit(dev_X, dev_y) 
print(sklearn.metrics.balanced_accuracy_score(test_y, model.predict(test_X)))

# "One vs. Rest" ROC curves 

In [ ]:
data_dev = pd.read_csv('/Users/mariamb/Desktop/BrainPower/brainpower/data/split_data/dev.csv')
data_test = pd.read_csv('/Users/mariamb/Desktop/BrainPower/brainpower/data/split_data/test.csv')
metadata = pd.read_csv('/Users/mariamb/Desktop/BrainPower/brainpower/data/metadata.csv')

data_dev, data_test = add_biomarkers_as_features(metadata, data_dev, data_test)


for i in feature_options: 
    roc_curves_one_vs_rest(data_dev, data_test, list_features=i)


# "One vs. One" ROC Curves

In [ ]:
for i in feature_options: 
    roc_curve_one_vs_one(data_dev, data_test, i) #change features u choose